## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
!pip install gmaps
import gmaps
import numpy as np

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv(r"C:\Users\DLH\Documents\Weather Database\WeatherPy_database.csv")
city_data_df.head()

,Unnamed: 0,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,aflu,dz,31.883867,1.459673,0.00,0,0,0.00,NaN
1,1,hambantota,lk,-0.916165,87.480602,75.70,87,{'all': 16},6.93,few clouds
2,2,norman wells,ca,73.344074,-119.794987,-9.40,83,{'all': 75},4.38,broken clouds
3,3,jamestown,sh,-28.164778,-10.134299,33.01,80,{'all': 1},17.65,clear sky
4,4,bluff,nz,-63.417944,172.551918,69.01,47,{'all': 0},3.00,clear sky


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("Enter minimum temperature: "))
max_temp = float(input("Enter maximum temperature: "))

Enter minimum temperature: 0
Enter maximum temperature: 100


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
filtered_data = city_data_df.loc[(city_data_df['Max Temp'] >= min_temp) &
                                 (city_data_df['Max Temp'] <= max_temp)]
filtered_data.head()

,Unnamed: 0,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,aflu,dz,31.883867,1.459673,0.00,0,0,0.00,NaN
1,1,hambantota,lk,-0.916165,87.480602,75.70,87,{'all': 16},6.93,few clouds
3,3,jamestown,sh,-28.164778,-10.134299,33.01,80,{'all': 1},17.65,clear sky
4,4,bluff,nz,-63.417944,172.551918,69.01,47,{'all': 0},3.00,clear sky
5,5,golyshmanovo,ru,56.269385,68.646473,13.75,95,{'all': 100},15.35,overcast clouds


In [5]:
# 4a. Determine if there are any empty rows.
np.where(pd.isnull(filtered_data))

(array([   0,   21,   23,   25,   34,   43,   51,   59,   60,   80,   88,
          89,  119,  122,  129,  147,  149,  159,  169,  174,  185,  201,
         203,  227,  237,  273,  291,  300,  304,  322,  327,  355,  378,
         384,  389,  392,  405,  420,  433,  445,  453,  462,  466,  480,
         482,  510,  521,  526,  541,  547,  560,  567,  591,  615,  637,
         648,  655,  661,  666,  668,  669,  672,  677,  683,  721,  754,
         760,  762,  767,  780,  786,  792,  801,  802,  826,  830,  840,
         841,  855,  866,  872,  879,  916,  924,  937,  969,  987, 1020,
        1026, 1047, 1072, 1086, 1089, 1093, 1130, 1142, 1147, 1154, 1155,
        1158, 1160, 1162, 1163, 1166, 1172, 1173, 1183, 1226, 1241, 1242,
        1245, 1253, 1255, 1256, 1257, 1258, 1273, 1279, 1283, 1284, 1286,
        1289, 1299, 1302, 1310, 1311, 1313, 1329, 1353, 1359, 1397, 1413,
        1429, 1445, 1446, 1456, 1472, 1474, 1507, 1518, 1521, 1522, 1539,
        1549, 1584, 1593, 1601, 1608, 

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
filtered_data = filtered_data.dropna()
np.where(pd.isnull(filtered_data))

(array([], dtype=int64), array([], dtype=int64))

In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = filtered_data[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,hambantota,lk,75.70,few clouds,-0.916165,87.480602,
3,jamestown,sh,33.01,clear sky,-28.164778,-10.134299,
4,bluff,nz,69.01,clear sky,-63.417944,172.551918,
5,golyshmanovo,ru,13.75,overcast clouds,56.269385,68.646473,
6,busselton,au,61.00,clear sky,-72.407366,94.567831,
7,codrington,ag,58.30,broken clouds,23.913131,-55.172555,
8,bonthe,sl,78.94,broken clouds,0.683460,-19.842215,
9,bluff,nz,69.01,clear sky,-79.169903,161.410871,
10,nsanje,mw,75.76,overcast clouds,-17.156385,35.810946,
11,guerrero negro,mx,64.67,clear sky,19.645037,-123.328752,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for i, row in hotel_df.iterrows():
    print(i)
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    long = row["Lng"]
    params["location"] = f"{lat},{long}"

    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
   
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[i, "Hotel Name"] = hotels["results"][0]["name"]
    except(IndexError):
        pass

1
3
4
5
6
7
8
9
10
11
12
13
14
15
17
18
19
21
22
23
25
27
29
30
31
32
33
34
35
36
38
39
40
41
42
43
44
45
47
48
49
50
51
52
53
55
56
57
58
59
60
61
65
66
67
68
71
72
73
74
75
76
77
78
79
80
81
82
84
85
86
88
89
90
91
92
93
94
97
98
99
100
101
103
104
105
106
107
108
109
110
111
112
114
115
116
118
119
120
121
122
123
124
125
126
128
129
132
133
135
136
137
138
139
140
142
143
144
145
147
148
149
150
151
152
153
155
156
158
159
160
161
163
165
166
167
168
169
170
171
172
173
176
177
178
179
180
183
184
185
186
188
189
190
191
193
194
195
196
197
198
199
200
202
204
206
207
208
209
210
211
212
213
215
216
217
218
221
222
223
225
228
229
230
231
233
234
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
254
255
257
259
260
261
262
263
265
267
268
269
270
273
274
276
277
278
279
280
281
282
283
286
287
288
289
290
291
292
294
295
296
297
298
299
300
302
303
304
305
306
307
308
310
311
312
313
314
315
316
317
318
319
321
322
323
324
326
327
328
331
334
335
336
337
338
339
3

In [9]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df.head()
clean_hotel_df = hotel_df.dropna()


In [10]:
# 8a. Create the output File (CSV)
output_data_file = 'WeatherPy_vacation.csv'
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")
clean_hotel_df.head()

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,hambantota,lk,75.70,few clouds,-0.916165,87.480602,
3,jamestown,sh,33.01,clear sky,-28.164778,-10.134299,
4,bluff,nz,69.01,clear sky,-63.417944,172.551918,
5,golyshmanovo,ru,13.75,overcast clouds,56.269385,68.646473,
6,busselton,au,61.00,clear sky,-72.407366,94.567831,


In [11]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>City</dt><dd>{Country}</dd>
<dt>Current Description</dt><dd>{Max Temp}</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [12]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# 11b. Display the figure
!jupyter nbextension enable --py gmaps
!jupyter nbextension enable --py widgetsnbextension
fig

Enabling notebook extension jupyter-gmaps/extension...
      - Validating: ok
Enabling notebook extension jupyter-js-widgets/extension...
      - Validating: ok


Figure(layout=FigureLayout(height='420px'))